In [ ]:
# 自然言語でデータベースを検索する
# 参考：LangChainを使って自然言語でデータベースを検索する（Azure編）（https://zenn.dev/cannoteat/articles/ae6040bfaa52d4）

# 方法１（リレーショナルデータベース）
# chatgptで入力からsql文を生成し、sql文を実行し、そこからランダムに返す

# 方法2 （ベクトルデータベース）
# ベクトルデータで入力をベクトル化し、ベクトルデータベースの中から類似度が高いものを返す
# 参考：OpenAIのEmbeddings APIのベクトルを使って検索を行う（https://dev.classmethod.jp/articles/search-with-openai-embeddings/）

In [6]:
# 方法0 簡易な方法
import openai
import os, json
from pprint import pprint

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

user_input = "奈良で有名な絵画が展示される展示会"


system = "あなたは博識で優秀な博物館のキュレーターです。"
exhibition_input = """以下は開催中の展示物の一覧です。以下の質問についてこの表を使用して答えてください。
展示番号    館名	都道府県	展示名	会期(始)	会期(終)	公式サイトURL	説明文	分類番号
01  四日市市立博物館	三重県	開館30周年記念　特別展 立原位貫　〜想像力から創造力へ〜	2023/9/23	2023/11/5	https://www.city.yokkaichi.mie.jp/museum/info/pg329.html	四日市ゆかりの木版画家・立原位貫（1951-2015）は、独学で江戸期浮世絵の魅力を現代に蘇らせ、さらにはオリジナルでも粋で個性的な作品を創り出しました。大英博物館に収蔵されるなど、世界に誇れる画家の足跡を辿り、その面白さ、美しさを紹介します。	73,09,721
03  パラミタ  ミュージアム	三重県	つながる琳派スピリット　神坂雪佳展	2023/10/6	2023/11/26	https://paramitamuseum.com/plan/next_exhibition.html#plan01	近代京都において、琳派を手本として創作活動を行った神坂雪佳(1866～1942)。京に生まれた雪佳は、はじめ四条派の画法を学びますが、新しい時代にふさわしい工芸図案の必要性を認識し、図案家の岸光景に師事します。押し寄せる西洋文化を意識しながらも、雪佳は装飾芸術である「琳派」に傾倒し、図案家、画家として活躍しました。雪佳が手掛けたデザインは、染織、陶芸、漆芸から室内装飾や庭園まで実に多面的です。これは、雪佳が敬愛した本阿弥光悦や尾形光琳の柔軟な活動に倣うものといえるでしょう。琳派は、江戸初期の本阿弥光悦、俵屋宗達(生没年不詳)にはじまり、江戸中期の光琳やその弟・尾形乾山、江戸後期の酒井抱一らによって私淑という形で受け継がれてきました。平安以来の美意識をそなえた装飾性豊かな作風で、絵画や工芸といった領域を越え、さまざまな意匠を生み出してきた琳派は、近代以降も多くの分野にわたって影響を与え続けています。本展では、雪佳が手本とした琳派の美の潮流を本阿弥光悦や尾形光琳らの名品にたどるとともに、古典と近代的発想を融合させ、美術と意匠の二つの分野を自在に往来した近代琳派・神坂雪佳の多彩な世界をご覧いただきます。	21,12,70,721,727,750,757
04  パラミタ  ミュージアム	三重県	Sasa Adairコレクション 20世紀巨匠の版画達展 シャガール、ピカソ、ダリからロックウェルまで	2023/12/1	2024/1/29	https://paramitamuseum.com/plan/schedule.html	マティス、ブラックからビュッフェの時代まで、彼らが信頼を寄せたパリの版画工房や出版社の協力で制作されたリトグラフ。アメリカンイラストレーションの黄金時代を築いたパリッシュ、古き良きアメリカを象徴するロックウェルによる雑誌の表紙絵。それらの版画約130点を展観いたします。ヨーロッパとアメリカの“版画達”を集めて帰国した「Sasa Adairコレクション」を公開。誰もが聞いたことがある作家、教科書で見たことがある作品をみつけながら、20世紀の芸術をお楽しみください。	25,23,73,74,727
06  桑名市博物館	三重県	特別企画展 三方領知替200年　行田市・桑名市・白河市友好都市締結25周年記念合同企画展「武門の遺産ー徳川家を支えた忍・桑名・白河ー」	2023/10/28	2023/11/26	https://www.city.kuwana.lg.jp/hakubutukan/bunka/bunkashisetsu/2023calendar.html	文政6年（1823）に忍藩主阿部正権が白河へ、白河藩主松平定永が桑名へ、桑名藩主松平忠堯が忍へ国替えとなる三方領知替（さんぽうりょうちがえ）が行われました。この国替えが縁となり、平成10年（1998）に行田市・桑名市・白河市の三市は友好都市を締結しました。令和5年（2023）は三方領知替200年、友好都市締結25周年という節目の年となります。三市の歴史・文化的魅力を広く市民や各市を訪れる人々に伝える絶好の機会であることから、三市の所蔵する「おたから」を一堂に会する展覧会を実施し、三市の文化財の魅力を各市において紹介し、今後の観光交流を推進していく上での一助とします。	21,09,721,728,750
07  桑名市博物館	三重県	企画展「BE BRAVE ―ヴィアティン三重、2023年の軌跡―」	2023/12/9	2023/12/17	https://www.city.kuwana.lg.jp/hakubutukan/bunka/bunkashisetsu/2024calendar.html	2023年のシーズンを振り返る企画展を開催いたします。	78
09  桑名市博物館	三重県	春季企画展「刀剣アラカルト」	2024/3/9	2024/5/12	https://www.city.kuwana.lg.jp/hakubutukan/bunka/bunkashisetsu/2026calendar.html	館蔵の刀剣を中心に、一振一振異なる味わいをご紹介します。一つとして同じもののない刀剣の魅力をご堪能ください。	21,756
10  公益財団法人松伯美術館	奈良県	文化勲章受章・卒寿記念特別展 上村淳之展	2023/10/14	2023/12/24	https://www.kintetsu-g-hd.co.jp/culture/shohaku/exhibition/exhibition.html	令和５年、当館の館長・上村淳之は卒寿を迎えます。淳之は、幼いころから、父 松篁が飼っていた小鳥たちとふれあい、美大三年生までの課題制作をおえると、専ら花鳥画を描くようになります。「鳥が好きだからテーマになり、テーマになるから鳥の数も増えてゆく」と述べるように、「唳禽荘」（自宅・アトリエ）で多くの鳥と共に暮らし、絵筆を握ってきました。その業績により、平成25年（2013）に文化功労者となり、令和４年（2022）文化勲章を受章しました。 東洋的感性の減退や現代の日本画について真摯に警告を発し、さまざまな鳥の姿を通じて、自然の叡智を示し、未来へのメッセージを送り続ける上村淳之の姿を紹介します。	
17  公益財団法人大和文華館	奈良県	文字を愛でる　―経典・文学・手紙から―	2024/2/23	2024/4/7	https://www.kintetsu-g-hd.co.jp/culture/yamato/exhibition/	中国や日本の書の文化は、書体や書風を多彩に展開し、文字を芸術の域にまで昇華させました。当館が所蔵する経典や文学作品、手紙を通して、美しい文字の世界へご案内します。	

"""

response_format = """回答は以下の形式で行ってください。
[{
    "exhibition_number": {展示番号},
    "exhibition_name": {展示名},
    "exhibition_reason": {その展示を選んだ理由}
},
~~~
~~~
~~~
{
    "exhibition_number": {展示番号},
    "exhibition_name": {展示名},
    "exhibition_reason": {その展示を選んだ理由}
}]
"""


question = "以下に与えられる入力をもとにおすすめの展示を選んでください。\n" + user_input

messages = [
    {"role": "system", "content": system},
    {"role": "system", "content": exhibition_input},
    {"role": "system", "content": response_format},
    {"role": "user", "content": question}
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages
)
result = response.choices[0]["message"]["content"].strip()
result_json = json.loads(result)
pprint(result_json)


[{'exhibition_name': '文化勲章受章・卒寿記念特別展 上村淳之展',
  'exhibition_number': 10,
  'exhibition_reason': '上村淳之展は、奈良県の公益財団法人松伯美術館で開催される特別展です。上村淳之は令和５年に卒寿を迎え、文化勲章も受章した著名な画家です。展示では、上村淳之の花鳥画や鳥をテーマにした作品を紹介し、彼の独自の感性と鳥の姿を通じて自然の叡智や未来へのメッセージを表現しています。奈良で有名な絵画を鑑賞したい方におすすめです。'}]


In [4]:
# 方法2：ベクトルデータベースの検索
#!pip install openai[embeddings]

import openai
from openai.embeddings_utils import cosine_similarity
import os,json

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

# データベースの読み込み
with open('embedding.json') as f:
    INDEX = json.load(f)

# これが検索用の文字列
QUERY = '幕末　明治維新　'

# 検索用の文字列をベクトル化
query = openai.Embedding.create(
    model='text-embedding-ada-002',
    input=QUERY
)

query = query['data'][0]['embedding']

# 総当りで類似度を計算
results = map(
    lambda i: {
        'number': i['number'],
        'name': i['name'],
        'raw': i['raw'],
        'embedding_text': i['embedding_text'],
        # ここでクエリと各文章のコサイン類似度を計算
        'similarity': cosine_similarity(i['embedding'], query)
        },
    INDEX
)
# コサイン類似度で降順（大きい順）にソート
results = sorted(results, key=lambda i: i['similarity'], reverse=True)

# 以下で結果を表示
print(f"Query: {QUERY}")
print("Rank: Title Similarity")
for i, result in enumerate(results):
    print(f'{i+1}: {result["number"]} {result["raw"][1]} {result["name"]} {result["similarity"]}')

print("====Best Doc====")
print(f'number: {results[0]["number"]}')
print(f'name: {results[0]["name"]}')

print("====Worst Doc====")
print(f'number: {results[-1]["number"]}')
print(f'name: {results[-1]["name"]}')

Query: 幕末　明治維新　
Rank: Title Similarity
1: 97 兵庫県 豊岡の幕末・維新 0.8327428635833457
2: 38 京都府 桂小五郎と幾松～広戸文庫史料～ 0.8314227548028784
3: 43 京都府 京都画壇の青春―栖鳳、松園につづく新世代たち 0.8300131992446413
4: 47 京都府 ［2023 冬期］コレクションルーム 特集「昭和前期の日本画と古典」 0.8248217768752268
5: 53 京都府 白沙村荘秋季展　2023　「名品よ、再び。」 0.8186602980813632
6: 30 京都府 開館45周年記念「定本 樂歴代」 0.8163027994532871
7: 137 和歌山県 律令国家成立前夜 0.8159230676415918
8: 117 奈良県 文化勲章受章・卒寿記念特別展 上村淳之展 0.8151392027465109
9: 74 大阪府 特別展「高槻城主列伝」 0.8147870161898606
10: 120 奈良県 近代の日本画〈日本画展示室〉 0.8128966000280872
11: 1 三重県 開館30周年記念　特別展 立原位貫　〜想像力から創造力へ〜 0.8117550174009304
12: 8 三重県 新春企画展「松平定信と源氏物語」【特集陳列】〈山〉と〈月〉の日本画／刀剣セレクション3 ー桑博ドラゴンズー 0.8116178184972742
13: 59 京都府 写真・新聞で振り返る昭和28年災害 0.810884771565663
14: 2 三重県 開館30周年記念　特別企画展　昭和のくらし　昭和の一隅 0.8107098662595704
15: 85 兵庫県 イギリス切手 1952-2022 0.8094860848248728
16: 77 大阪府 作家の道へ―司馬遼太郎と「近代説話」 0.8089719667539637
17: 123 奈良県 特別企画展　やまと絵のこころ 0.8089549919262328
18: 119 奈良県 松本竣介、駒井哲郎から舟越桂まで－戦後日本の洋画・版画－〈洋画展示室〉 0.8087976522937105
19: 55 京都府 表装の愉しみ －ある表具師のものがたり 

In [9]:
# 方法2.5：質問文を元chatgptがQueryを作成＆ベクトルデータベースの検索(谷口改変)　それなりに動きました
#!pip install openai[embeddings]

import openai
from openai.embeddings_utils import cosine_similarity
import os,json

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY


exhibition_count = 5


# データベースの読み込み
with open('embedding.json') as f:
    INDEX = json.load(f)

user_input = "モナリザとかはどこに行けば見れますか？"
print(f"User_Input: {user_input}")
init_role_describe = """あなたは優秀な検索エンジンです。あなたは来客者様からの入力を元に、博物館内の展示を探す必要があります。そのための検索用の単語を出力しなくてはいけません。
出力は単語のみを出力してください。説明文などは必要ありません。各単語を","で区切って出力してください。
"""

message = f"""「{user_input}」
という来客者様からの文章を元に展示を調べなくてはいけません。
検索するための単語を、複数個あげてください。10個ほどあればよいでしょう。
とにかく関連しそうなワードを挙げてください。
この来客者様がどんなものが好きそうなのか、想像してみてください。
"""

messages = [
    {"role": "system", "content": init_role_describe},
    {"role": "user", "content": message}
]

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages
)

# これが検索用の文字列
QUERY  = response.choices[0]["message"]["content"].strip()
print(f"Query: {QUERY}")

# 検索用の文字列をベクトル化
query = openai.Embedding.create(
    model='text-embedding-ada-002',
    input=QUERY
)

query = query['data'][0]['embedding']

# 総当りで類似度を計算
results = map(
    lambda i: {
        'number': i['number'],
        'name': i['name'],
        'raw': i['raw'],
        'embedding_text': i['embedding_text'],
        # ここでクエリと各文章のコサイン類似度を計算
        'similarity': cosine_similarity(i['embedding'], query)
        },
    INDEX
)
# コサイン類似度で降順（大きい順）にソート
results = sorted(results, key=lambda i: i['similarity'], reverse=True)

# 以下で結果を表示


print("Rank: Title Similarity")
top_suggested_museums = ""
for i, result in enumerate(results[:exhibition_count]):
    print(f'{i+1}: {result["number"]}  {result["raw"][1]} {result["raw"][0]}  {result["name"]} {result["similarity"]}')

# print("====Best Doc====")
# print(f'number: {results[0]["number"]}')
# print(f'name: {results[0]["name"]}')

# print("====Worst Doc====")
# print(f'number: {results[-1]["number"]}')
# print(f'name: {results[-1]["name"]}')

exhibition_text = "\n".join([f"{index+1}. " + exhibition["embedding_text"] for index, exhibition in enumerate(results[:exhibition_count])])
#exhibition_text = "\n".join([f"{index+1}. " + exhibition["raw"][1] + " "+exhibition["raw"][0] + " "+exhibition["name"] for index, exhibition in enumerate(results[:exhibition_count])])
third_msg = f"""
上記のキーワードをもとに検索した結果、以下の{exhibition_count}個の展示が検索エンジンから提案されました。
{exhibition_text}
この{exhibition_count}個の展示からひとつ来客者様に対して提案しようと思うのですが、
これらの情報と来客者様からの入力をもとに、来客者様の入力に対してフレンドリーな返答をしてください。
来客者様の考えを読み取り、来客者様が聞いて楽しいような返答を心がけてください。
出力は以下の形式で行ってください。日本語で返答してください。
{{
    "exhibition_number": {{展示番号}},
    "exhibition_name": {{展示名}},
    "exhibition_reason": {{その展示を選んだ理由}}
}}
"""

print(third_msg)
messages = [
    {"role": "system", "content": init_role_describe},
    {"role": "user", "content": message},
    {"role":"assistant","content":QUERY},
    {"role": "user", "content": third_msg}
]

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages
)

# 最終的な出力
output  = response.choices[0]["message"]["content"].strip()

print(output)

User_Input: 彼女と博物館デートに行きたいと思っています。彼女も僕も民俗学に興味があります。
Query: 民俗学,伝統,良俗,風俗,民族衣装,祭り,伝説,民話,生活道具,民家
Rank: Title Similarity
1: 151  大阪府 国立民族学博物館  常設展 0.8470222130082268
2: 137  和歌山県 和歌山市立博物館  歴史を語る道具たち 0.8368315529473707
3: 108  兵庫県 小野市立好古館  お堂の道具あれこれー小野市文化財悉皆調査速報展2ー 0.8286482725033608
4: 27  京都府 京都市学校歴史博物館  きらめきはぐくむ 京都の学校と伝統工芸――西陣織・京友禅・京焼―― 0.8276026623779067
5: 111  奈良県 公益財団法人名勝依水園・寧楽美術館  やきもの用語実見展　虫喰い、口紅ってなんだろう?? 0.8272390238690346

上記のキーワードをもとに検索した結果、以下の5個の展示が検索エンジンから提案されました。
1. 大阪府にある国立民族学博物館で行われる常設展は2023/1/1から2023/12/31まで開催されます。その内容は本館における展示は、地域展示と通文化展示からなっています。地域展示では、世界をオセアニア、アメリカ、ヨーロッパ、アフリカ、それに日本を含むアジア各地域に分け、オセアニアを出発して東回りに世界を一周し、最後に日本にたどり着く構成をとっています。日本の文化を世界各地の文化との関連の中で理解できるように配慮したものです。民博では、創設以来、世界の民族文化に優劣はなく、すべて等しい価値をもつという認識にもとづいて、展示をつくり上げてきました。それぞれの文化に見られる違いは、人類の営みの豊かな多様性を示すものとして展示されています。また、世界の人びとの暮らしがよくわかるように、衣食住などの生活用品を中心とした展示になっているのも特徴のひとつです。。分類：風俗習慣．民俗学．民族学
2. 和歌山県にある和歌山市立博物館で行われる歴史を語る道具たちは2024/1/16から2024/3/3まで開催されます。その内容は民具や農具を「くらしの道具」・「遊びと勉強」・「あかりの道具」・「食事の道具」・「はかる道具」・「あたたまる道具」

In [2]:
results

[{'number': 119,
  'name': '空海\u3000KŪKAI ―密教のルーツとマンダラ世界',
  'raw': ['奈良国立博物館',
   '奈良県',
   '空海\u3000KŪKAI ―密教のルーツとマンダラ世界',
   '2024/4/13',
   '2024/6/9',
   'https://www.narahaku.go.jp/exhibition/special/special_exhibition/202404_kukai/',
   '空海の生誕1250年を記念して、奈良国立博物館の総力を挙げた展覧会を開催します。「虚空尽き、衆生尽き、涅槃尽きなば、わが願いも尽きむ。」（『性霊集』巻第八）空海が人々を救うためにたどり着いたのは密教でした。本展では、空海がもたらした密教の国際的なルーツを辿るとともに、空海が伝えたマンダラの世界を展示室に展開し、その偉業を顕彰します。\u3000（この世の全ての物が消滅し、仏法の世界が尽きるまで、私は人々が救われることを願い続ける）',
   ''],
  'embedding_text': '奈良県にある奈良国立博物館で行われる空海\u3000KŪKAI ―密教のルーツとマンダラ世界は2024/4/13から2024/6/9まで開催されます。その内容は空海の生誕1250年を記念して、奈良国立博物館の総力を挙げた展覧会を開催します。「虚空尽き、衆生尽き、涅槃尽きなば、わが願いも尽きむ。」（『性霊集』巻第八）空海が人々を救うためにたどり着いたのは密教でした。本展では、空海がもたらした密教の国際的なルーツを辿るとともに、空海が伝えたマンダラの世界を展示室に展開し、その偉業を顕彰します。\u3000（この世の全ての物が消滅し、仏法の世界が尽きるまで、私は人々が救われることを願い続ける）。分類：未分類',
  'similarity': 0.8215975388397244},
 {'number': 21,
  'name': '三重のさかな\u3000伊勢エビ',
  'raw': ['鳥羽市立海の博物館',
   '三重県',
   '三重のさかな\u3000伊勢エビ',
   '2023/7/15',
   '2023/11/23',
   'http://www.umihaku.com/